# Introduction

Even if you're new to Connect Four, you've likely developed several game-playing strategies.  In this tutorial, you'll learn to use a **heuristic** to share your knowledge with the agent.  

# Game trees

As a human player, how do you think about how to play the game?  How do you weigh alternative moves?

You likely do a bit of forecasting.  For each potential move, you predict what your opponent is likely to do in response, along with how you'd then respond, and what the opponent is likely to do then, and so on.  Then, you choose the move where you think you're most likely to win.

We can formalize this idea and represent all possible outcomes in a **(complete) game tree**.  

<center>
<img src="https://i.imgur.com/EZKHxyy.png"><br/>
</center>

The game tree represents each possible move (by player and opponent), starting with an empty board.  Then, the first row shows all possible moves the red player can make.  Next, we record each move the yellow player can make in response, and so on, until each branch reaches the end of the game.  (_The game tree for Connect Four is quite large, so we show only a small preview in the image above_.)

Once we can see every way the game can possibly end, it can help us to pick the move where we are most likely to win.

# Heuristics

The complete game tree for Connect Four has over [4 trillion](https://oeis.org/A212693) different boards!  So in practice, our agent can only work with a small subset when planning a move. 

To make sure the incomplete tree is still useful to the agent, we will use a **heuristic** (or **heuristic function**).  The heuristic assigns scores to different game boards, where we estimate that boards with higher scores are more likely to result in the agent winning the game.  You will design the heuristic based on your knowledge of the game.

For instance, one heuristic that might work reasonably well for Connect Four looks at each grid position of four spots in a (horizontal, vertical, or diagonal) line and assigns:
- **100 points** if the agent has four discs in a row (the agent won), 
- **1 point** if the agent filled three spots, and the remaining spot is empty (the agent wins if it fills in the empty spot), 
- **-10 points** if the opponent filled three spots, and the remaining spot is empty (the opponent wins by filling in the empty spot), and
- **-100 points** if the opponent has four discs in a row (the opponent won).

This is also represented in the image below.

<center>
<img src="https://i.imgur.com/UlUYhgr.png" width=50%><br/>
</center>

So, how exactly will the agent use the heuristic?  Consider it's the agent's turn, and it's trying to plan a move for the game board shown at the top of the figure below.  There are seven possible moves (one for each column).

<center>
<img src="https://i.imgur.com/lGfMNBK.png" width=100%><br/>
</center>

The heuristic assigns the first board (where the agent plays in column 0) a score of 20.  The second board is assigned a score of 10, and so on.  The first board receives the highest score, and so the agent will select this move.  It's also the best outcome for the agent, since it has a guaranteed win in just one more move.  

The heuristic works really well for this specific example, since it matches the best move with the highest score!  In general, if you're not sure how to design your heuristic (i.e., how to score different game states, or which scores to assign to different conditions), often the best thing to do is to simply take an initial guess and then play against your agent.  This will let you identify specific cases when your agent makes bad moves, which you can then fix by modifying the heuristic.

# Code

Our **one-step lookahead** agent will:
- use the heuristic to assign a score to each possible valid move, and
- select the move that gets the highest score.  (_If multiple moves get the high score, we select one at random._)

"One-step lookahead" refers to the fact that the agent looks only one step (or move) into the future, instead of deeper in the game tree.  To help with this agent, we define several useful functions in the code cell below.  

In [ ]:
#$HIDE_INPUT$
import random
import numpy as np

In [ ]:
# Calculates value of heuristic if agent drops piece in selected column
def score_move(grid, col, mark, config):
    next_grid = drop_piece(grid, col, mark, config)
    num_threes = count_windows(next_grid, 3, mark, config)
    num_fours = count_windows(next_grid, 4, mark, config)
    num_threes_opp = count_windows(next_grid, 3, mark%2+1, config)
    score = num_threes - 1e3*num_threes_opp + 1e6*num_fours
    return score

# Gets board at next step if agent drops piece in selected column
def drop_piece(grid, col, mark, config):
    next_grid = grid.copy()
    for row in range(config.rows-1, -1, -1):
        if next_grid[row][col] == 0:
            break
    next_grid[row][col] = mark
    return next_grid

# Helper function for score_move: checks if window satisfies heuristic conditions
def check_window(window, num_discs, piece, config):
    return (window.count(piece) == num_discs and window.count(0) == config.inarow-num_discs)
    
# Helper function for score_move: counts number of windows satisfying specified heuristic conditions
def count_windows(grid, num_discs, piece, config):
    num_windows = 0
    # horizontal
    for row in range(config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[row, col:col+config.inarow])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # vertical
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns):
            window = list(grid[row:row+config.inarow, col])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # positive diagonal
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # negative diagonal
    for row in range(config.inarow-1, config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    return num_windows

- `score_move()` calculates the value of the heuristic for the board that results from playing a move in the selected column (in `col`) in the current board (in `grid`).
  - The  

The one-step lookahead agent is defined in the next code cell.

In [ ]:
# One-step lookahead agent
def agent1(obs, config):
    # Get list of valid moves
    valid_moves = [c for c in range(config.columns) if obs.board[c] == 0]
    # Convert the board to a 2D grid
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    # Use the heuristic to assign a score to each possible board in the next step
    scores = dict(zip(valid_moves, [score_move(grid, col, obs.mark, config) for col in valid_moves]))
    # Get a list of columns (moves) that maximize the heuristic
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
    # Select at random from the maximizing columns
    return random.choice(max_cols)

Next, the agent plays a game against a random agent.

In [ ]:
import kaggle_simulations as ks

# Create the game environment
env = ks.make("connectx")

# Two random agents play one game round
env.run([agent1, "random"])

# Show the game
env.render(mode="ipython")

# Your turn

Continue to the exercise to **[improve the heuristic](#$NEXT_NOTEBOOK_URL$)**.